## Gale-Shapley Algorithm

- **Two-sided Matching Problem**  
  - Two sets (e.g. men, women) each with preferences over the other.  
  - A matching is stable if no pair would both rather match with each other than their current partners.

- **Gale–Shapley Algorithm**  
  - Men (proposers) iteratively propose to their most-preferred available woman.  
  - Women (reviewers) tentatively accept their best current proposal and reject the rest.  
  - Continues until everyone is matched.  
  - Produces a stable matching (no blocking pairs).  

- **Evaluation Metrics**  
  - Average rank of matches (for men, for women).  
  - Number of blocking pairs (stability measure).  
  - Proposer-optimal vs receiver-optimal comparisons.  

In [1]:
import numpy as np

def gale_shapley(pref_m, pref_w):
    """
    pref_m: list of lists, where pref_m[m] is the ordered preference for man m
    pref_w: list of lists, where pref_w[w] is the ordered preference for woman w
    Returns: matching, a list of length #men with the index of woman matched to each man
    """
    n = len(pref_m)
    # Next woman to propose to for each man
    next_proposal = [0]*n  
    # Current engagements: woman_engaged[w] = man matched to w (or -1 if none)
    woman_engaged = [-1]*n
    # Free men queue
    free_men = list(range(n))

    # Convert woman's preference into "rank" for quick comparisons
    # rank_w[w][m] = rank of man m in w’s preference
    rank_w = []
    for w in range(n):
        rank = [0]*n
        for r, m in enumerate(pref_w[w]):
            rank[m] = r
        rank_w.append(rank)

    while free_men:
        m = free_men.pop(0)
        # Man m proposes to his next available woman
        w = pref_m[m][next_proposal[m]]
        next_proposal[m] += 1
        current_m = woman_engaged[w]
        if current_m == -1:
            # If woman w is free, engage
            woman_engaged[w] = m
        else:
            # If w prefers m over her current match, switch
            if rank_w[w][m] < rank_w[w][current_m]:
                woman_engaged[w] = m
                free_men.append(current_m)
            else:
                # Woman stays with current_m; m remains free
                free_men.append(m)

    # Build final matching man->woman
    matching = [-1]*n
    for w, m in enumerate(woman_engaged):
        matching[m] = w
    return matching

def random_matching(n):
    """ Returns a random matching for n men and n women. """
    perm = np.random.permutation(n)
    return perm.tolist()

def heuristic_matching(pref_m):
    """
    Simple heuristic: match each man with his top choice if possible, else next, etc. 
    This is naive and not necessarily stable.
    """
    n = len(pref_m)
    matched_women = set()
    matching = [-1]*n
    for m in range(n):
        for w in pref_m[m]:
            if w not in matched_women:
                matching[m] = w
                matched_women.add(w)
                break
    return matching

def average_rank(matching, pref_lists):
    """
    matching: list where matching[m] = w
    pref_lists: list of lists, pref_lists[m] = ordered preference for m
    Returns the average rank of matched partners for men or women (depending on pref_lists).
    """
    ranks = []
    for m, w in enumerate(matching):
        # rank = index of w in m's preference list
        ranks.append(pref_lists[m].index(w) + 1)
    return np.mean(ranks)

def count_blocking_pairs(matching_men, matching_women, pref_m, pref_w):
    """
    matching_men[m] = w matched with man m
    matching_women[w] = m matched with woman w
    pref_m, pref_w: preference lists
    Returns the number of blocking pairs.
    """
    n = len(matching_men)
    blocks = 0
    for m in range(n):
        w = matching_men[m]
        for w_alt in pref_m[m]:
            if w_alt == w:
                break
            # Check if w_alt also prefers m to her current match
            m_alt = matching_women[w_alt]
            # If w_alt is not matched, skip (or treat as no blocking)
            if m_alt < 0:
                continue
            # If man m prefers w_alt more than w,
            # and w_alt prefers m more than her current match,
            # then (m, w_alt) is a blocking pair
            if (pref_m[m].index(w_alt) < pref_m[m].index(w) and
                pref_w[w_alt].index(m) < pref_w[w_alt].index(m_alt)):
                blocks += 1
    return blocks

# ---- Run simulation ----
np.random.seed(42)
n = 6
# Generate random preferences
men_pref = [np.random.permutation(n).tolist() for _ in range(n)]
women_pref = [np.random.permutation(n).tolist() for _ in range(n)]

# Gale-Shapley
gs_match = gale_shapley(men_pref, women_pref)
gs_inv = [-1]*n
for m, w in enumerate(gs_match):
    gs_inv[w] = m

# Random
rnd_match = random_matching(n)
rnd_inv = [-1]*n
for m, w in enumerate(rnd_match):
    rnd_inv[w] = m

# Heuristic
heu_match = heuristic_matching(men_pref)
heu_inv = [-1]*n
for m, w in enumerate(heu_match):
    heu_inv[w] = m

# Evaluate
print("Gale-Shapley:")
print("  Average rank (men):", average_rank(gs_match, men_pref))
print("  Average rank (women):", average_rank(gs_inv, women_pref))
print("  Blocking pairs:", count_blocking_pairs(gs_match, gs_inv, men_pref, women_pref))

print("Random:")
print("  Average rank (men):", average_rank(rnd_match, men_pref))
print("  Average rank (women):", average_rank(rnd_inv, women_pref))
print("  Blocking pairs:", count_blocking_pairs(rnd_match, rnd_inv, men_pref, women_pref))

print("Heuristic:")
print("  Average rank (men):", average_rank(heu_match, men_pref))
print("  Average rank (women):", average_rank(heu_inv, women_pref))
print("  Blocking pairs:", count_blocking_pairs(heu_match, heu_inv, men_pref, women_pref))

Gale-Shapley:
  Average rank (men): 1.5
  Average rank (women): 2.6666666666666665
  Blocking pairs: 0
Random:
  Average rank (men): 3.6666666666666665
  Average rank (women): 4.333333333333333
  Blocking pairs: 11
Heuristic:
  Average rank (men): 2.0
  Average rank (women): 3.1666666666666665
  Blocking pairs: 2
